In [6]:
import pandas as pd
import torch
from ray import train,tune
import matplotlib.pyplot as plt

## Pertubration d'une ligne de métro
On va essayer de trouver les moment où le metro rencontre des problèmes.

Voici les cas: 
- Il y a trop de demande, même en condition normale, la capacité des rames et les fréquences d'arrivées d'arrivent plus à gérer.
    - On défini une station 'cible' (ex : Charpennes). On doit pouvoir recouper le nombre de personne totale arrivant ou sortant de la station cible avec :
        - L'historique d'entrée à la station cible.
        - L'historique des entrée des stations en direction de notre 'cible'.
        - La distance entre chaque station.
    - Pour étendre la vision du problème, on peut considérer le nombre de personne arrivant à des stations TC autour de notre 'cible'.
        - Si on a les validations des Tram et Bus en direction de 'cible' (ensemble S). Qu'on a les ids (donc on peut "tracker" une personne"), on pourrait inférer la proportion d'usagers qui au départ des stations de S, fini par prendre le métro ?  
            - Peut être que cette proportion est relativement constante ? Quelque soit les jours, les heures, événément ... ? Et alors à t-dt on pourrait dire "attention grande demande arrivant à la cible à t" ?  

- Le métro est coupé sur un court terme : Incident sur la voie, coupure d'électricité ... Il y a toujours des entrées, mais plus personne ne sort
    - On doit donc être capable de repérer ces épisodes par analyse des entrée / sortie
    - On sait donc que lors d'un tel incident, le nombre de personne validant = nombre de personne qui va être dans la prochaine rame. Problème identique sur toute les rames touchées de ligne. 
    - Pour le métro, il n'y a peut être aucun intérêt à trouver ces événements. Mais pour les stations TC adjacentes ? Peut être que lors de ce type d'événement, on enregistre 5% ou 10% de demande en plus sur les stations adjacentes ? Sur les Vélo'V ? On pourrait donc améliorer notre compréhension de mobilité des usagers (ex pour des simulations).

- Le métro est coupé sur un long terme : Les portes sont ouvertes en continue, il n'y a n'y sortie ni entrée.
    - Idem que précédement, mais cette fois-ci 100% des usagers soit se reportent sur d'autres TC, soit rentre chez eux. Sachant qu'on peut avoir une bonne estimation du nombre d'entrée de métro estimé à ce moment là (comme trafic hyper périodic).

## Data : 
Soit 16Mars - 31Mai 2019 avec Netmob et Validation Individuelles
Soit Novembre 2019 - Mars 2020 Avec Validation Individuelle  et Subway 15 min


In [ ]:
# Validation Individuelles :                         [J,F,M,A,M,J,J,A,S,O,N,D],[J,F,M, , , , , , , , , ],[J,F,M, , , , , , , , , ]
# Subway 15 Min :   Novembre 2019 - Mai 2021         [ , , , , , , , , , ,N,D],[J,F,M,A,M,J,J,A,S,O,N,D],[J,F,M,A,M, , , , , , , ]
# NetMob 15 Min :                                   [ , ,M,A,M, , , , , , , ],[ , , , , , , , , , , , ],[ , , , , , , , , , , , ]

In [12]:
metro_15min = pd.read_csv('data/intensite_de_deplacement_metro_15_min.csv')



(0, 1)

In [16]:
metro_15min.iloc[0],metro_15min.iloc[-1]

(Unnamed: 0    2021-01-01 00:00:00
 in                           69.0
 out                         152.0
 Name: 0, dtype: object,
 Unnamed: 0    2021-03-31 00:00:00
 in                          134.0
 out                         230.0
 Name: 8544, dtype: object)